# Agentic system

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "./"
sys.path.append(os.path.abspath(module_path))

## 1. Create Bedrock client

In [3]:
from pprint import pprint
from termcolor import colored
from utils import bedrock
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-5-V-2-Sonnet': 'anthropic.claude-3-5-sonnet-20241022-v2:0',
 'Claude-V3-5-V-2-Sonnet-CRI': 'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
 'Claude-V3-7-Sonnet-CRI': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Comma

## 2. LLM 정의

In [5]:
from utils.bedrock import bedrock_model
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm = bedrock_model(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
    #model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
    bedrock_client=boto3_bedrock,
    stream=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    inference_config={
        'maxTokens': 8192*3,
        'stopSequences': ["\n\nHuman"],
        'temperature': 0.01,
    }
)

## 3. Agentic system for MBTI Identificaton

In [7]:
from datetime import datetime
import pytz



import base64
import matplotlib.pyplot as plt
import io
import json
import pprint

# import traceback
from PIL import Image
# from termcolor import colored

from textwrap import dedent
from utils.bedrock import bedrock_utils
from typing import TypedDict#, Optional, List

from src.genai_anaysis import llm_call
from src.prompt import module_prompt

from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

from utils.common_utils import retry
# from botocore.exceptions import ClientError, ConnectionError, ConnectTimeoutError, ReadTimeoutError

In [8]:
def _get_message_from_string(role, string, imgs=None):
        
        message = {
            "role": role,
            "content": []
        }
        
        if imgs is not None:
            for img in imgs:
                img_message = {
                    "image": {
                        "format": 'png',
                        "source": {"bytes": img}
                    }
                }
                message["content"].append(img_message)
        
        message["content"].append({"text": dedent(string)})

        return message

def _message_format(role, message):

        if role == "user":
             message_format = {
                "role": "user",
                "content": [{"text": dedent(message)}]
            }
        elif role == "assistant":
            
            message_format = {
                "role": "assistant",
                'content': [{'text': dedent(message)}]
            }

        return message_format
    
    
def _png_to_bytes(file_path):
    """Convert a PNG file to binary data and base64 string.

    Args:
        file_path: Path to the PNG file.

    Returns:
        tuple: (binary_data, base64_string) or error message.
    """
    try:
        with open(file_path, "rb") as image_file:
            # Read file in binary mode
            binary_data = image_file.read()

            # Encode binary data to base64
            base64_encoded = base64.b64encode(binary_data)

            # Decode bytes to string
            base64_string = base64_encoded.decode('utf-8')

            return binary_data, base64_string

    except FileNotFoundError:
        print ("Error: 파일을 찾을 수 없습니다.")
    except Exception as e:
        print (f"Error: {str(e)}")

In [9]:
tool_list = [
    {
        "toolSpec": {
            "name": "message_notify_user",
            "description": "Send a message to user without requiring a response. Use for acknowledging receipt of messages, providing progress updates, reporting task completion, or explaining changes in approach.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "text": {
                            "type": "string",
                            "description": "Message text to display to user"
                        },
                        "attachments": {
                            "anyOf": [
                                {"type": "string"},
                                {"type": "array", "items": {"type": "string"}}
                            ],
                            "description": "(Optional) List of attachments to show to user, can be file paths or URLs"
                        }
                    },
                    "required": ["text"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "message_ask_user",
            "description": "Ask user a question and wait for response. Use for requesting clarification, asking for confirmation, or gathering additional information.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "text": {
                            "type": "string",
                            "description": "Question text to present to user"
                        },
                        "attachments": {
                            "anyOf": [
                                {"type": "string"},
                                {"type": "array", "items": {"type": "string"}}
                            ],
                            "description": "(Optional) List of question-related files or reference materials"
                        },
                        "suggest_user_takeover": {
                            "type": "string",
                            "enum": ["none", "browser"],
                            "description": "(Optional) Suggested operation for user takeover"
                        }
                    },
                    "required": ["text"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "file_read",
            "description": "Read file content. Use for checking file contents, analyzing logs, or reading configuration files.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "file": {
                            "type": "string",
                            "description": "Absolute path of the file to read"
                        },
                        "start_line": {
                            "type": "integer",
                            "description": "(Optional) Starting line to read from, 0-based"
                        },
                        "end_line": {
                            "type": "integer",
                            "description": "(Optional) Ending line number (exclusive)"
                        },
                        "sudo": {
                            "type": "boolean",
                            "description": "(Optional) Whether to use sudo privileges"
                        }
                    },
                    "required": ["file"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "file_write",
            "description": "Overwrite or append content to a file. Use for creating new files, appending content, or modifying existing files.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "file": {
                            "type": "string",
                            "description": "Absolute path of the file to write to"
                        },
                        "content": {
                            "type": "string",
                            "description": "Text content to write"
                        },
                        "append": {
                            "type": "boolean",
                            "description": "(Optional) Whether to use append mode"
                        },
                        "leading_newline": {
                            "type": "boolean",
                            "description": "(Optional) Whether to add a leading newline"
                        },
                        "trailing_newline": {
                            "type": "boolean",
                            "description": "(Optional) Whether to add a trailing newline"
                        },
                        "sudo": {
                            "type": "boolean",
                            "description": "(Optional) Whether to use sudo privileges"
                        }
                    },
                    "required": ["file", "content"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "file_str_replace",
            "description": "Replace specified string in a file. Use for updating specific content in files or fixing errors in code.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "file": {
                            "type": "string",
                            "description": "Absolute path of the file to perform replacement on"
                        },
                        "old_str": {
                            "type": "string",
                            "description": "Original string to be replaced"
                        },
                        "new_str": {
                            "type": "string",
                            "description": "New string to replace with"
                        },
                        "sudo": {
                            "type": "boolean",
                            "description": "(Optional) Whether to use sudo privileges"
                        }
                    },
                    "required": ["file", "old_str", "new_str"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "file_find_in_content",
            "description": "Search for matching text within file content. Use for finding specific content or patterns in files.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "file": {
                            "type": "string",
                            "description": "Absolute path of the file to search within"
                        },
                        "regex": {
                            "type": "string",
                            "description": "Regular expression pattern to match"
                        },
                        "sudo": {
                            "type": "boolean",
                            "description": "(Optional) Whether to use sudo privileges"
                        }
                    },
                    "required": ["file", "regex"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "file_find_by_name",
            "description": "Find files by name pattern in specified directory. Use for locating files with specific naming patterns.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "path": {
                            "type": "string",
                            "description": "Absolute path of directory to search"
                        },
                        "glob": {
                            "type": "string",
                            "description": "Filename pattern using glob syntax wildcards"
                        }
                    },
                    "required": ["path", "glob"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "shell_exec",
            "description": "Execute commands in a specified shell session. Use for running code, installing packages, or managing files.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "string",
                            "description": "Unique identifier of the target shell session"
                        },
                        "exec_dir": {
                            "type": "string",
                            "description": "Working directory for command execution (must use absolute path)"
                        },
                        "command": {
                            "type": "string",
                            "description": "Shell command to execute"
                        }
                    },
                    "required": ["id", "exec_dir", "command"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "shell_view",
            "description": "View the content of a specified shell session. Use for checking command execution results or monitoring output.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "string",
                            "description": "Unique identifier of the target shell session"
                        }
                    },
                    "required": ["id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "shell_wait",
            "description": "Wait for the running process in a specified shell session to return. Use after running commands that require longer runtime.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "string",
                            "description": "Unique identifier of the target shell session"
                        },
                        "seconds": {
                            "type": "integer",
                            "description": "Wait duration in seconds"
                        }
                    },
                    "required": ["id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "shell_write_to_process",
            "description": "Write input to a running process in a specified shell session. Use for responding to interactive command prompts.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "string",
                            "description": "Unique identifier of the target shell session"
                        },
                        "input": {
                            "type": "string",
                            "description": "Input content to write to the process"
                        },
                        "press_enter": {
                            "type": "boolean",
                            "description": "Whether to press Enter key after input"
                        }
                    },
                    "required": ["id", "input", "press_enter"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "shell_kill_process",
            "description": "Terminate a running process in a specified shell session. Use for stopping long-running processes or handling frozen commands.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "string",
                            "description": "Unique identifier of the target shell session"
                        }
                    },
                    "required": ["id"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_view",
            "description": "View content of the current browser page. Use for checking the latest state of previously opened pages.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {}
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_navigate",
            "description": "Navigate browser to specified URL. Use when accessing new pages is needed.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "url": {
                            "type": "string",
                            "description": "Complete URL to visit. Must include protocol prefix."
                        }
                    },
                    "required": ["url"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_restart",
            "description": "Restart browser and navigate to specified URL. Use when browser state needs to be reset.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "url": {
                            "type": "string",
                            "description": "Complete URL to visit after restart. Must include protocol prefix."
                        }
                    },
                    "required": ["url"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_click",
            "description": "Click on elements in the current browser page. Use when clicking page elements is needed.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "index": {
                            "type": "integer",
                            "description": "(Optional) Index number of the element to click"
                        },
                        "coordinate_x": {
                            "type": "number",
                            "description": "(Optional) X coordinate of click position"
                        },
                        "coordinate_y": {
                            "type": "number",
                            "description": "(Optional) Y coordinate of click position"
                        }
                    }
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_input",
            "description": "Overwrite text in editable elements on the current browser page. Use when filling content in input fields.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "index": {
                            "type": "integer",
                            "description": "(Optional) Index number of the element to overwrite text"
                        },
                        "coordinate_x": {
                            "type": "number",
                            "description": "(Optional) X coordinate of the element to overwrite text"
                        },
                        "coordinate_y": {
                            "type": "number",
                            "description": "(Optional) Y coordinate of the element to overwrite text"
                        },
                        "text": {
                            "type": "string",
                            "description": "Complete text content to overwrite"
                        },
                        "press_enter": {
                            "type": "boolean",
                            "description": "Whether to press Enter key after input"
                        }
                    },
                    "required": ["text", "press_enter"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_move_mouse",
            "description": "Move cursor to specified position on the current browser page. Use when simulating user mouse movement.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "coordinate_x": {
                            "type": "number",
                            "description": "X coordinate of target cursor position"
                        },
                        "coordinate_y": {
                            "type": "number",
                            "description": "Y coordinate of target cursor position"
                        }
                    },
                    "required": ["coordinate_x", "coordinate_y"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_press_key",
            "description": "Simulate key press in the current browser page. Use when specific keyboard operations are needed.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "key": {
                            "type": "string",
                            "description": "Key name to simulate (e.g., Enter, Tab, ArrowUp), supports key combinations (e.g., Control+Enter)."
                        }
                    },
                    "required": ["key"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_select_option",
            "description": "Select specified option from dropdown list element in the current browser page. Use when selecting dropdown menu options.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "index": {
                            "type": "integer",
                            "description": "Index number of the dropdown list element"
                        },
                        "option": {
                            "type": "integer",
                            "description": "Option number to select, starting from 0."
                        }
                    },
                    "required": ["index", "option"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_scroll_up",
            "description": "Scroll up the current browser page. Use when viewing content above or returning to page top.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "to_top": {
                            "type": "boolean",
                            "description": "(Optional) Whether to scroll directly to page top instead of one viewport up."
                        }
                    }
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_scroll_down",
            "description": "Scroll down the current browser page. Use when viewing content below or jumping to page bottom.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "to_bottom": {
                            "type": "boolean",
                            "description": "(Optional) Whether to scroll directly to page bottom instead of one viewport down."
                        }
                    }
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_console_exec",
            "description": "Execute JavaScript code in browser console. Use when custom scripts need to be executed.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "javascript": {
                            "type": "string",
                            "description": "JavaScript code to execute. Note that the runtime environment is browser console."
                        }
                    },
                    "required": ["javascript"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "browser_console_view",
            "description": "View browser console output. Use when checking JavaScript logs or debugging page errors.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "max_lines": {
                            "type": "integer",
                            "description": "(Optional) Maximum number of log lines to return."
                        }
                    }
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "info_search_web",
            "description": "Search web pages using search engine. Use for obtaining latest information or finding references.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Search query in Google search style, using 3-5 keywords."
                        },
                        "date_range": {
                            "type": "string",
                            "enum": ["all", "past_hour", "past_day", "past_week", "past_month", "past_year"],
                            "description": "(Optional) Time range filter for search results."
                        }
                    },
                    "required": ["query"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "deploy_expose_port",
            "description": "Expose specified local port for temporary public access. Use when providing temporary public access for services.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "port": {
                            "type": "integer",
                            "description": "Local port number to expose"
                        }
                    },
                    "required": ["port"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "deploy_apply_deployment",
            "description": "Deploy website or application to public production environment. Use when deploying or updating static websites or applications.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "type": {
                            "type": "string",
                            "enum": ["static", "nextjs"],
                            "description": "Type of website or application to deploy."
                        },
                        "local_dir": {
                            "type": "string",
                            "description": "Absolute path of local directory to deploy."
                        }
                    },
                    "required": ["type", "local_dir"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "make_manus_page",
            "description": "Make a Manus Page from a local MDX file.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "mdx_file_path": {
                            "type": "string",
                            "description": "Absolute path of the source MDX file"
                        }
                    },
                    "required": ["mdx_file_path"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "idle",
            "description": "A special tool to indicate you have completed all tasks and are about to enter idle state.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {}
                }
            }
        }
    }
]

tool_config = {
    "tools": tool_list,
    #"toolChoice": {
    #    "tool": {
    #        "name": "summarize_email"
    #    }
    #}
}

참고자료: https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/tool-use-examples.html

In [10]:
def message_notify_user(text):
    print ("===This is tool for message_notify_user===")
    print ("text:", text)
    return text


# 도구 사용을 처리하는 함수
def process_tool_result(tool):
    """도구 사용 요청을 처리하고 결과를 반환하는 함수"""
    tool_result = {}
    
    if tool['name'] == 'message_notify_user':
        try:
            text = message_notify_user(tool['input']['text'])
            tool_result = {
                "toolUseId": tool['toolUseId'],
                "content": [{"json": {"text": text}}]
            }
        except Exception as err:
            tool_result = {
                "toolUseId": tool['toolUseId'],
                #"content": [{"json": {"text": str(err)}}],
                "content": [{"text": str(err)}],
                "status": 'error'
            }
    else:
        
        tool_result = {
                "toolUseId": tool['toolUseId'],
                "content": [{"json": {"text": "done!"}}],
            }
        
    # 여기에 다른 도구 처리 추가 가능
    
    tool_result_message = {
        "role": "user",
        "content": [
            {
                "toolResult": tool_result
            }
        ]
    }
    return tool_result_message

In [11]:
messages = []

llm.stream = True
llm_caller = llm_call(llm=llm, verbose=False, tracking=False)

system_prompts = module_prompt    
system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

# UTC 시간대로 현재 시간 가져오기
utc_now = datetime.now(pytz.UTC)
# ISO 8601 형식으로 포맷팅 (YYYY-MM-DDThh:mm:ssZ)
timestamp = utc_now.strftime("%Y-%m-%dT%H:%M:%SZ")


user_prompts = dedent(
    '''
    <event type="Message" timestamp={timestamp}>
    <sender>user_123</sender>
    <content>{content}</content>
    </event>
    '''
    
)
context = {
    "timestamp": timestamp,
    #"content": "주어진 이미지로 부터 인사이트를 추출해줘",
    #"content": "주어진 데이터를 분석하고 보고서 작성해줘",
    # "content":
    #     '''
    #     'data.csv',
    #     주어진 데이터를 분석하고 보고서 작성해줘. 필요하다면 todo리스트도 만들고 보고서에 차트도 넣어줘.
    #     '''
    "content":'''
    ,cont_line_seq,churn_yn,thinq_entr,breakdown_b0617,funtion,survey_ans_count,survey_service_avg_score,rental_sales_channel,call_a070621,survey_nps_avg_score,discount_combine_type,survey_service_min_score,call_a050101,water_d0065,call_a070625,manager_age_max,svc_normal_count,manager_age_avg,stop_sum,call_a060401,water_d0048,call_a050104,color,survey_nps_under_score_count,discount_churn_cal,manager_skill_avg,buy_gc_yn,call_ib_total,manager_change_count,manager_skill_min,call_install_ask,visit_time_diff_avg,visit_n_count,call_a070624,house_price,call_a070615,survey_manager_avg_score,discount_standard_sum,discount_combine_sum,call_a060601,call_a070623,retal_5year_fee,as_water_paid,r_cr_yn,r_gc_yn,breakdown_b0882,as_water_product_problem,discount_mistake,discount_commitment_max,lgbest_sales_channel,obs_contract_status,lge_entr_yn,cont_expy_count,contract_dtl_type,call_a030101,age_nm,rental_usage_days,call_a040101,payment_type,stair_fee_yn,visit_time_diff_max,customer_price,careship_total_contract,call_a070614,obs_water_plp,contract_user_yn,rental_mktg_agree_yn,gender_nm,as_water_survey_avg,time
0,14025423,N,1,0,온수/냉수/정수,3,4.333333333333333,하이프라자(H),0,,N,4.0,1,0,0,62,0,52.77272727272727,0,7,0,0,샤이니로즈,0.0,0,2.933333333333333,N,4,5,1,1,27.75,2,0,,0,5.0,0,0,40,0,22900,0,N,N,0,0,0,30900,BEST SHOP(H),0,Y,1,신규,0,65.0,1248,1,카드이체,Y,42.0,1771684.0,0,0,0,Y,Y,여자,,1686882135
1,12767520,Y,0,0,온수/냉수/정수,0,,온라인케어솔루션전문점,0,,N,,0,0,0,56,0,56.0,0,2,0,0,샤이니로즈,,0,1.75,N,2,1,0,1,22.75,0,0,,0,,0,0,28,0,32900,0,N,N,1,0,0,32900,렌탈전문점,0,Y,0,신규,0,45.0,1181,1,은행이체,N,31.0,1520682.0,0,0,0,Y,N,여자,,1686882135
2,14044817,N,0,0,냉수/정수,6,4.666666666666667,온라인케어솔루션전문점,0,3.0,N,4.0,0,0,0,58,1,52.65,0,12,0,0,샤이니로즈,2.0,0,2.1333333333333333,N,3,4,0,0,25.444444444444443,0,0,,0,5.0,0,0,35,0,19900,0,N,N,0,0,24900,24900,렌탈전문점,16,Y,0,신규,0,42.0,1242,0,카드이체,Y,35.0,,0,0,0,Y,N,남자,,1686882135
3,14372633,N,0,0,온수/냉수/정수,3,4.666666666666667,온라인케어솔루션전문점,0,3.0,U,4.0,0,0,0,56,0,56.0,0,11,0,0,화이트,1.0,0,2.2857142857142856,Y,0,1,1,0,25.125,1,0,,0,5.0,34900,30000,57,0,26900,0,N,N,0,0,0,33900,렌탈전문점,7,Y,0,신규,0,33.0,1147,0,은행이체,Y,41.0,,0,0,0,Y,N,남자,,1686882135
4,14143155,N,0,0,온수/냉수/정수,2,5.0,온라인케어솔루션전문점,0,,N,5.0,0,0,0,56,0,45.64,0,10,0,0,샤이니로즈,0.0,0,2.769230769230769,N,1,7,0,0,22.875,5,0,,0,5.0,0,0,32,0,22900,0,N,N,1,0,0,27900,렌탈전문점,0,Y,0,신규,0,33.0,1220,0,은행이체,Y,36.0,1692663.0,0,0,0,Y,N,여자,,1686882135
5,14893052,N,0,0,온수/냉수/정수,0,,하이프라자(H),0,,N,,0,0,0,44,0,43.523809523809526,0,3,0,0,실버,,0,3.083333333333333,N,0,3,0,0,22.0,1,0,,0,,0,0,32,0,22900,0,N,N,0,0,0,30900,BEST SHOP(H),0,N,0,신규,0,35.0,1017,0,카드이체,Y,35.0,1486572.0,0,0,0,Y,Y,여자,,1686882135
6,14598288,N,0,0,온수/냉수/정수,5,4.833333333333333,하이프라자(H),0,9.0,N,4.0,0,0,0,46,0,46.0,0,6,0,0,실버,0.0,0,9.384615384615383,N,1,1,8,0,23.5,0,0,,0,4.875,0,0,25,0,26900,0,N,N,0,0,0,34900,BEST SHOP(H),0,Y,0,신규,0,67.0,1077,0,카드이체,Y,43.0,1197424.0,0,0,0,Y,Y,여자,,1686882135
7,14923591,N,1,0,냉수/정수,0,,전문점(J),0,,N,,0,0,0,57,0,50.4,0,2,0,0,실버,,0,1.25,Y,1,3,0,0,29.375,0,0,,0,,0,0,30,0,20900,0,N,N,0,0,0,30900,BEST SHOP(J),0,Y,0,신규,0,47.0,1010,0,카드이체,Y,54.0,1624661.0,0,0,0,Y,Y,남자,,1686882135
8,14027386,N,0,0,냉수/정수,3,5.0,온라인케어솔루션전문점,0,10.0,N,5.0,0,0,0,58,0,58.0,0,3,0,0,실버,0.0,0,5.133333333333334,N,1,1,3,0,23.4,0,0,31100.0,0,5.0,0,0,44,0,19900,0,N,N,0,0,0,29900,렌탈전문점,0,Y,0,신규,0,46.0,1247,0,카드이체,Y,35.0,2108034.0,0,0,0,Y,N,여자,,1686882135
9,14551871,N,1,0,온수/냉수/정수,0,,하이케어솔루션(B2C),0,,N,,0,0,0,56,0,47.47619047619048,0,3,0,0,실버,,0,2.2,N,0,10,0,0,29.22222222222222,0,0,,0,,0,0,22,0,26900,0,N,N,0,0,0,36900,렌탈전문점,0,N,0,신규,0,51.0,1089,0,카드이체,Y,57.0,1928392.0,0,0,0,Y,N,남자,,1686882135

주어진 데이터를 분석하고 보고서 작성해줘. 필요하다면 todo리스트도 만들고 보고서에 차트도 넣어줘.
'''
}
user_prompts = user_prompts.format(**context)

img_bytes_1, _ = _png_to_bytes("1.png")
img_bytes_2, _ = _png_to_bytes("2.png")
img_bytes_3, _ = _png_to_bytes("3.png")
img_bytes_4, _ = _png_to_bytes("4.png")


message = _get_message_from_string(role="user", string=user_prompts, imgs=[])
#message = _get_message_from_string(role="user", string=user_prompts, imgs=[img_bytes_1, img_bytes_2, img_bytes_3, img_bytes_4])
messages.append(message)



# 반복 대화 처리를 위한 설정
MAX_TURNS = 15  # 무한 루프 방지용 최대 턴 수
turn = 0
final_response = False

# 도구 사용이 종료될 때까지 반복
while not final_response and turn < MAX_TURNS:
    turn += 1
    print(f"\n--- 대화 턴 {turn} ---")
    #print ("messages", messages)
    
    # 모델에 메시지 전송
    resp, ai_message = llm_caller.invoke(
        messages=messages,
        system_prompts=system_prompts,
        tool_config=tool_config,
        enable_reasoning=False,
        reasoning_budget_tokens=8192
    )
    
    # 모델 응답을 대화에 추가
    #print ("ai_message", ai_message)
    messages.append(ai_message)
    print(f"응답 상태: {resp['stop_reason']}")
    
    # 도구 사용 요청 확인
    if resp["stop_reason"] == "tool_use":
        print("모델이 도구 사용을 요청했습니다.")
        
        tool_requests_found = False
        
        # 응답에서 모든 도구 사용 요청 처리
        for content in ai_message['content']:
            if 'toolUse' in content:
                tool = content['toolUse']
                tool_requests_found = True
                
                print(f"요청된 도구: {tool['name']}")
                print(f"입력 데이터: {tool['input']}")
                
                # 도구 요청 처리 및 결과 메시지 생성
                tool_result_message = process_tool_result(tool)
                #print ("tool_result_message", tool_result_message)
                
                # 결과 메시지를 대화에 추가
                messages.append(tool_result_message)
                print(f"도구 실행 결과를 대화에 추가했습니다.")
        
        # 도구 요청이 없으면 루프 종료
        if not tool_requests_found:
            print("도구 요청을 찾을 수 없습니다.")
            final_response = True
    else:
        # 도구 사용이 요청되지 않았으면 최종 응답으로 간주
        final_response = True
        print("최종 응답을 받았습니다.")

print("\n=== 대화 완료 ===")
print("최종 응답:", resp)
print("메시지:", ai_message)




--- 대화 턴 1 ---
enable_reasoning False
self.llm.additional_model_request_fields None
self.llm.inference_config {'maxTokens': 24576, 'stopSequences': ['\n\nHuman'], 'temperature': 0.01}
I'll help you analyze this dataset and create a comprehensive report with visualizations. Let me start by acknowledging your request and setting up a plan.{"text": "안녕하세요! 제공해주신 데이터를 분석하고 보고서를 작성해드리겠습니다. 먼저 데이터를 파일로 저장하고 분석 계획을 세운 후, 차트와 함께 상세한 보고서를 작성하겠습니다. 잠시만 기다려주세요."}
Stop reason: tool_use
응답 상태: tool_use
모델이 도구 사용을 요청했습니다.
요청된 도구: message_notify_user
입력 데이터: {'text': '안녕하세요! 제공해주신 데이터를 분석하고 보고서를 작성해드리겠습니다. 먼저 데이터를 파일로 저장하고 분석 계획을 세운 후, 차트와 함께 상세한 보고서를 작성하겠습니다. 잠시만 기다려주세요.'}
===This is tool for message_notify_user===
text: 안녕하세요! 제공해주신 데이터를 분석하고 보고서를 작성해드리겠습니다. 먼저 데이터를 파일로 저장하고 분석 계획을 세운 후, 차트와 함께 상세한 보고서를 작성하겠습니다. 잠시만 기다려주세요.
도구 실행 결과를 대화에 추가했습니다.

--- 대화 턴 2 ---
enable_reasoning False
self.llm.additional_model_request_fields None
self.llm.inference_config {'maxTokens': 24576, 'stopSequences': ['\n\

ReadTimeoutError: AWSHTTPSConnectionPool(host='bedrock-runtime.us-west-2.amazonaws.com', port=443): Read timed out.